In [1]:
import os
import mlflow
import pandas as pd
import psycopg

In [2]:
os.environ["DB_DESTINATION_HOST"] = os.getenv("DB_DESTINATION_HOST")
os.environ["DB_DESTINATION_PORT"] = os.getenv("DB_DESTINATION_PORT")
os.environ["DB_DESTINATION_NAME"] = os.getenv("DB_DESTINATION_NAME")
os.environ["DB_DESTINATION_USER"] = os.getenv("DB_DESTINATION_USER")
os.environ["DB_DESTINATION_PASSWORD"] = os.getenv("DB_DESTINATION_PASSWORD")

In [3]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.environ["DB_DESTINATION_HOST"], 
    "port": os.environ["DB_DESTINATION_PORT"],
    "dbname": os.environ["DB_DESTINATION_NAME"],
    "user": os.environ["DB_DESTINATION_USER"],
    "password": os.environ["DB_DESTINATION_PASSWORD"],
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [4]:
#сохранить список названий столбцов в строку
for i in df.columns.values:
    print(i)



id
customer_id
begin_date
end_date
type
paperless_billing
payment_method
monthly_charges
total_charges
internet_service
online_security
online_backup
device_protection
tech_support
streaming_tv
streaming_movies
gender
senior_citizen
partner
dependents
multiple_lines
target


In [5]:
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(','.join(df.columns.values.tolist()))
   

In [6]:
','.join(df.columns.values.tolist())

'id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target'

In [7]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
		# посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}
    print(column_stat)
    stats.update(column_stat)


stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max() # посчитайте максимальное значение в колонке
stats["monthly_charges_mean"] = df["monthly_charges"].mean() # посчитайте среднее значение в колонке
stats["monthly_charges_median"] = df["monthly_charges"].median() # посчитайте медианное значение в колонке
stats["total_charges_min"] = df["total_charges"].min() # посчитайте минимальное значение в колонке
stats["total_charges_max"] = df["total_charges"].max() # посчитайте максимальное значение в колонке
stats["total_charges_mean"] = df["total_charges"].mean() # посчитайте среднее значение в колонке
stats["total_charges_median"] = df["total_charges"].median() # посчитайте медианное значение в колонке
stats["unique_customers_number"] = df["customer_id"].nunique() # посчитайте кол-во уникальных id
stats["end_date_nan"] = df["end_date"].isna().sum() # посчитайте кол-во пустых строк в колонке


{'type_Month-to-month': 3875, 'type_Two year': 1695, 'type_One year': 1473}
{'paperless_billing_Yes': 4171, 'paperless_billing_No': 2872}
{'internet_service_Fiber optic': 3096, 'internet_service_DSL': 2421}
{'online_security_No': 3498, 'online_security_Yes': 2019}
{'online_backup_No': 3088, 'online_backup_Yes': 2429}
{'device_protection_No': 3095, 'device_protection_Yes': 2422}
{'tech_support_No': 3473, 'tech_support_Yes': 2044}
{'streaming_tv_No': 2810, 'streaming_tv_Yes': 2707}
{'streaming_movies_No': 2785, 'streaming_movies_Yes': 2732}
{'gender_Male': 3555, 'gender_Female': 3488}
{'senior_citizen_0': 5901, 'senior_citizen_1': 1142}
{'partner_No': 3641, 'partner_Yes': 3402}
{'dependents_No': 4933, 'dependents_Yes': 2110}
{'multiple_lines_No': 3390, 'multiple_lines_Yes': 2971}
{'target_0': 5174, 'target_1': 1869}


In [8]:
df["end_date"].isna().sum()

5174

In [9]:
#посчитайте кол-во пустых строк в колонке
stats


{'type_Month-to-month': 3875,
 'type_Two year': 1695,
 'type_One year': 1473,
 'paperless_billing_Yes': 4171,
 'paperless_billing_No': 2872,
 'internet_service_Fiber optic': 3096,
 'internet_service_DSL': 2421,
 'online_security_No': 3498,
 'online_security_Yes': 2019,
 'online_backup_No': 3088,
 'online_backup_Yes': 2429,
 'device_protection_No': 3095,
 'device_protection_Yes': 2422,
 'tech_support_No': 3473,
 'tech_support_Yes': 2044,
 'streaming_tv_No': 2810,
 'streaming_tv_Yes': 2707,
 'streaming_movies_No': 2785,
 'streaming_movies_Yes': 2732,
 'gender_Male': 3555,
 'gender_Female': 3488,
 'senior_citizen_0': 5901,
 'senior_citizen_1': 1142,
 'partner_No': 3641,
 'partner_Yes': 3402,
 'dependents_No': 4933,
 'dependents_Yes': 2110,
 'multiple_lines_No': 3390,
 'multiple_lines_Yes': 2971,
 'target_0': 5174,
 'target_1': 1869,
 'data_length': 7043,
 'monthly_charges_min': 18.25,
 'monthly_charges_max': 118.75,
 'monthly_charges_mean': 64.76169246059918,
 'monthly_charges_median': 70

In [10]:
df.to_csv("users_churn.csv", index=False)

In [11]:
# задаём название эксперимента и имя запуска для логирования в MLflow

EXPERIMENT_NAME = "churn_fio"
RUN_NAME = "data_check"

# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) # ваш код здесь

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id # ваш код здесь
    
    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats) # ваш код здесь
    
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact('columns.txt', "dataframe")  # ваш код здесь
    mlflow.log_artifact('users_churn.csv', "dataframe")  # ваш код здесь


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id) # ваш код здесь


# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert (run.info.status =='FINISHED')# ваш код здесь

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove('columns.txt') # ваш код здесь
os.remove('users_churn.csv') # ваш код здесьcsv

In [23]:

mlflow.search_experiments()

[<Experiment: artifact_location='file:///home/andrey/work/MLE/mle-mlflow/mlruns/477980246858346885', creation_time=1711612482269, experiment_id='477980246858346885', last_update_time=1711612482269, lifecycle_stage='active', name='churn_fio', tags={}>,
 <Experiment: artifact_location='file:///home/andrey/work/MLE/mle-mlflow/mlruns/306879251168284368', creation_time=1711611736572, experiment_id='306879251168284368', last_update_time=1711611736572, lifecycle_stage='active', name='churn_Andrey2', tags={}>,
 <Experiment: artifact_location='file:///home/andrey/work/MLE/mle-mlflow/mlruns/428528985635553178', creation_time=1711611719641, experiment_id='428528985635553178', last_update_time=1711611719641, lifecycle_stage='active', name='churn_Andrey1', tags={}>,
 <Experiment: artifact_location='file:///home/andrey/work/MLE/mle-mlflow/mlruns/320194216947062806', creation_time=1711609525482, experiment_id='320194216947062806', last_update_time=1711609525482, lifecycle_stage='active', name='churn_

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.unique_customers_number,metrics.total_charges_mean,metrics.total_charges_min,metrics.partner_No,...,metrics.streaming_tv_Yes,metrics.internet_service_Fiber optic,metrics.senior_citizen_1,metrics.dependents_No,metrics.senior_citizen_0,tags.mlflow.source.git.commit,tags.mlflow.source.name,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.source.type
0,08467bb39ab649bb9509056eb7e8d6c9,477980246858346885,FINISHED,file:///home/andrey/work/MLE/mle-mlflow/mlruns...,2024-03-28 07:55:57.959000+00:00,2024-03-28 07:55:58.005000+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,f1fb1b4bc5d5c8814c8275a7a71dc09119f0be96,/home/andrey/work/MLE/mle-mlflow/.venv_mlflow/...,andrey,data_check,LOCAL
1,89ef3a445f824616978ebd1e786bebcc,477980246858346885,FINISHED,file:///home/andrey/work/MLE/mle-mlflow/mlruns...,2024-03-28 07:54:42.308000+00:00,2024-03-28 07:54:42.328000+00:00,7043.0,2283.300441,18.8,3641.0,...,2707.0,3096.0,1142.0,4933.0,5901.0,f1fb1b4bc5d5c8814c8275a7a71dc09119f0be96,/home/andrey/work/MLE/mle-mlflow/.venv_mlflow/...,andrey,data_check,LOCAL
